# EX-situ GIWAXS

BL 11-3 SLAC - February 2025

This is a tutorial which explores how to use MDA to analyze ex-situ GIWAXS data. The data was collected at beamline 11-3 at SLAC in February 2025. The data is from a series of samples synthesised and prepared by Dr. Adam Marks and measured at SLAC by Dr. Arianna Magni. The samples are a series of polymer thin films with varying thicknesses.

## Import Packages and Classes

We will start by importing the necessary packages and classes.

In [ ]:
import pandas as pd
import plotly.express as px
from glob import glob

from Materials_Data_Analytics.experiment_modelling.giwaxs import Calibrator
from Materials_Data_Analytics.experiment_modelling.giwaxs import GIWAXSPixelImage

## Create Callibrator

Next we will create a callibration object.  This contains information about the beamline and the detector used for calculating diffraction patterns from the beamline images

In [ ]:
calibrator = Calibrator.from_poni_file('/path/to/calibration.poni')
calibrator

## Find the files and extract the values

Our next task is to read in the image files created by the beamline.  First, we will find all the files in a directory that corrospond to a pattern match. These will be all the beamline files we want to read in.


In [ ]:
tif_files = glob('/path/to/tiff/files/*.tif')
tif_files

Next,  we will turn these paths into a dataframe.  We will also extract the solution concentration from the name, as an indicator for the mass loading of the film. 

In [ ]:

concentration_map = {
    '5mgmL': 5,
    '10mgmL': 10,
    '2p5mgmL': 2.5,
}

file_data_frame = (pd
                   .DataFrame({'path': tif_files})
                   .assign(concentration = lambda x: [concentration_map[s.split('/')[-1].split('_')[2]] for s in x['path']])
                   )

file_data_frame

Each concentration has 3 images. This is often the case as the sample is moved around to get a better average of the diffraction pattern. The next bit of code will automatically adjust the positions of the images and average them together correctly to get a final, high resolution GIWAXS image. 

## Read in the files to create the GIWAXS Image

In the following part, we will group this dataframe by concentration. For each concentration, we will read the beamline images in to the .from_SLAC_BL11_3() consutrctor. This will create a GIWAXSPixelImage object. From this image object, we calculate a diffraction pattern.

In [ ]:

patterns = []

for name, df in file_data_frame.groupby('concentration'):

    files = df['path'].to_list()

    pattern = (GIWAXSPixelImage
               .from_SLAC_BL11_3(files)
               .get_giwaxs_pattern(calibrator=calibrator, pixel_q = 1024, pixel_chi = 360)
               )
    
    patterns.append(pattern)


data = (pd
        .DataFrame({'pattern': patterns, 'concentration': file_data_frame['concentration'].unique()})
        .sort_values('concentration')
        )

data

# Plots

## plot 2D maps

Here,  for each pattern we plot a 2D map of the diffraction pattern. This is a good way to check the quality of the data and to get familiar with the features. 

In [ ]:

for pattern, concentration in zip(data['pattern'], data['concentration']):

    figure = pattern.plot_reciprocal_map(intensity_lower_cuttoff = 1, 
                                         width=800, 
                                         height=500, 
                                         title=f'AM946 at concentration {concentration} mg/ml',
                                         template = 'plotly_dark')
    
    figure.show()


In [ ]:

for pattern, concentration in zip(data['pattern'], data['concentration']):

    figure = pattern.plot_polar_map(intensity_lower_cuttoff = 1, 
                                    width=800, 
                                    height=500, 
                                    title=f'AM946 at concentration {concentration} mg/ml',
                                    template = 'plotly_dark')
    
    figure.show()

## plot linecuts
#### out of plane

In [ ]:
data_plot = []

for pattern, concentration in zip(data['pattern'], data['concentration']):

    linecut_data = (pattern
                    .get_linecut(chi = (5,20))
                    .data
                    .assign(concentration = concentration)
                    .query('q < 2')
                    )
    
    data_plot.append(linecut_data)
    
data_plot = pd.concat(data_plot)

px.line(data_plot, x='q', y='intensity', color='concentration', title='Linecut of AM946 at different concentrations', template='presentation', width=900, height=500, log_y=True).show()


#### in plane

In [ ]:
data_plot = []

for pattern, concentration in zip(data['pattern'], data['concentration']):

    linecut_data = (pattern
                    .get_linecut(chi = (70,87))
                    .data
                    .assign(concentration = concentration)
                    .query('q < 2')
                    )
    
    data_plot.append(linecut_data)
    
data_plot = pd.concat(data_plot)

px.line(data_plot, x='q', y='intensity', color='concentration', title='Linecut of AM946 at different concentrations', template='presentation', width=900, height=500, log_y=True).show()

# fitting
### lamellar peak

In [ ]:

a = (a
     .assign(bins = pd.cut(a['q'], bins = 1000))
     .groupby('bins')
     .agg('mean')
     )


In [ ]:
initial_fitting_parameters = {'peak_center_value': 0.88,
                              'peak_sigma_max': 0.1,
                              'peak_center_min': 0.1,
                              'peak_center_max': 1,
                              'peak_amplitude_value': 100,
                              'bkg_intercept_value': 0,
                              'bkg_slope_value': 0}

In [ ]:
out_of_plane_lamellar_fits = []

for p, c in zip(data['pattern'], data['concentration']):

    linecut = p.get_linecut(chi = (5, 20))
    peak_data = linecut.data.query('q > 0.7 and q < 1.05')

    fit = (linecut
           .fit_linecut(peak_model = 'LorentzianModel',
                        background_model = 'LinearModel',
                        q_range = (0.7, 1.05),
                        initial_parameters = initial_fitting_parameters)
           .fit_results
           )
    
    peak_data['peak_fit'] = fit.eval_components()['peak_']
    peak_data['background_fit'] = fit.eval_components()['bkg_']
    peak_data['total_fit'] = peak_data['peak_fit'] + peak_data['background_fit']
    peak_data['concentration'] = c
    peak_data = peak_data.melt(id_vars=['q', 'chi', 'concentration'], var_name='fit_type', value_name='fit_value')

    out_of_plane_lamellar_fits.append(peak_data)

plot_data = pd.concat(out_of_plane_lamellar_fits)

px.line(plot_data, facet_row='concentration', x='q', y='fit_value', color='fit_type', title='Fitting of AM946 at different concentrations', template='presentation', width=600, height=900).show()